In [ ]:
! pip install -q datasets scikit-learn seaborn matplotlib joblib kagglehub plotly

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arshkon/linkedin-job-postings")
import os

os.listdir(path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df_posting = pd.read_csv(f'{path}/postings.csv')
df_posting.info()

In [ ]:
# Hiển thị 5 dòng đầu và 5 dòng ngẫu nhiên để có cái nhìn tổng quan
display(df_posting.head())
display(df_posting.sample(5))

In [ ]:
# Tính phần trăm missing value cho từng cột và sắp xếp giảm dần
missing_percent = df_posting.isnull().mean() * 100
print(missing_percent[missing_percent > 0].sort_values(ascending=False))

In [ ]:
# Kiểm tra phân phối của các cột quan trọng
cols_to_check = ['formatted_work_type', 'formatted_experience_level', 'remote_allowed', 'pay_period']

for col in cols_to_check:
    print(f"--- Phân phối giá trị của cột: {col} ---")
    print(df_posting[col].value_counts(dropna=False))
    print("\n")

In [ ]:
# Xem thống kê mô tả (mean, min, max, std) của các cột số
display(df_posting[['max_salary', 'min_salary', 'views', 'applies']].describe())

In [ ]:
import plotly.express as px
# --- 1. XỬ LÝ DỮ LIỆU (Giống bước trước) ---
df_clean = df_posting.copy()

# Xử lý remote_allowed và thời gian
df_clean['remote_allowed'] = df_clean['remote_allowed'].fillna(0).astype(int)
df_clean['listed_time'] = pd.to_datetime(df_clean['listed_time'], unit='ms')

# Chỉ lấy đơn vị tiền tệ USD
if 'currency' in df_clean.columns:
    df_clean = df_clean[df_clean['currency'].isin(['USD', pd.NA, None])]

# Tính lương trung bình và chuẩn hóa về Lương Năm
df_clean['avg_salary'] = (df_clean['min_salary'] + df_clean['max_salary']) / 2

def standardize_salary(row):
    salary = row['avg_salary']
    period = row['pay_period']
    if pd.isna(salary): return None
    if period == 'HOURLY': return salary * 40 * 52
    if period == 'MONTHLY': return salary * 12
    if period == 'WEEKLY': return salary * 52
    return salary

df_clean['yearly_salary'] = df_clean.apply(standardize_salary, axis=1)

# Lọc bỏ ngoại lai (Outliers)
df_clean = df_clean[
    (df_clean['yearly_salary'].isnull()) |
    ((df_clean['yearly_salary'] >= 15000) & (df_clean['yearly_salary'] <= 600000))
]

In [ ]:
# 1. Kiểm tra trùng lặp
print(f"Số lượng dòng trước khi khử trùng: {len(df_clean)}")

# Kiểm tra trùng job_id
dupes_id = df_clean.duplicated(subset=['job_id']).sum()
print(f"Số lượng job_id bị trùng: {dupes_id}")

# Kiểm tra trùng nội dung (Company + Title + Description) - trường hợp repost
dupes_content = df_clean.duplicated(subset=['company_name', 'title', 'description']).sum()
print(f"Số lượng bài đăng trùng nội dung: {dupes_content}")

# Xử lý: Giữ lại bài đăng mới nhất (nếu có listed_time)
df_clean = df_clean.sort_values('listed_time', ascending=False)
df_clean = df_clean.drop_duplicates(subset=['company_name', 'title', 'description'], keep='first')

print(f"Số lượng dòng sau khi khử trùng: {len(df_clean)}")

In [ ]:
# 2. Trích xuất Bang (State) từ cột location
# Logic: Lấy 2 ký tự cuối cùng nếu chúng viết hoa, giả định định dạng "City, ST"
def extract_state(location):
    if pd.isna(location): return 'Unknown'
    if ',' in location:
        state = location.split(',')[-1].strip()
        # Kiểm tra xem có phải mã bang 2 ký tự (VD: CA, NY, TX)
        if len(state) == 2 and state.isupper():
            return state
    return 'Other'

df_clean['job_state'] = df_clean['location'].apply(extract_state)

# Kiểm tra phân bố theo Bang
state_stats = df_clean['job_state'].value_counts().head(10)
print("Top 10 Bang có nhiều việc làm nhất:")
display(state_stats.to_frame())

In [ ]:
# 3. Trích xuất đặc trưng thời gian
df_clean['post_month'] = df_clean['listed_time'].dt.month
df_clean['post_day_name'] = df_clean['listed_time'].dt.day_name()
df_clean['post_hour'] = df_clean['listed_time'].dt.hour

# Thống kê lượng bài đăng theo ngày trong tuần
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = df_clean['post_day_name'].value_counts().reindex(day_order)

print("Phân bố bài đăng theo thứ trong tuần:")
display(day_counts.to_frame())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 4. Tìm Top 2-grams (Cụm 2 từ) xuất hiện nhiều nhất
def get_top_ngrams(corpus, n=None, top_k=10):
    vec = CountVectorizer(ngram_range=(n, n), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_k]

# Chỉ lấy mẫu 5000 dòng để chạy cho nhanh (Text mining tốn RAM)
sample_desc = df_clean['description'].dropna().sample(min(5000, len(df_clean)), random_state=42)

top_bigrams = get_top_ngrams(sample_desc, n=2, top_k=15)
print("Top 15 cụm từ (2 chữ) xuất hiện nhiều nhất trong mô tả công việc:")
df_bigrams = pd.DataFrame(top_bigrams, columns=['Phrase', 'Frequency'])
display(df_bigrams)

In [ ]:
# --- 1. THỐNG KÊ PHÂN PHỐI LƯƠNG (Dạng Bảng/JSON) ---
print("### 1. Thống kê mô tả Lương (Yearly Salary) ###")
# Hiển thị các chỉ số thống kê cơ bản
salary_stats = df_clean['yearly_salary'].describe().to_frame().T
display(salary_stats)

# Chia khoảng lương (Binning) để thấy phân phối tần suất
# Tạo 10 khoảng lương từ thấp đến cao
bins = list(range(0, int(df_clean['yearly_salary'].max()) + 20000, 20000))
salary_distribution = pd.cut(df_clean['yearly_salary'], bins=bins).value_counts().sort_index()

print("\n### Phân phối tần suất Lương (Frequency Table) ###")
# Chuyển thành DataFrame cho dễ đọc
dist_df = salary_distribution.reset_index()
dist_df.columns = ['Salary Range', 'Count']
dist_df['Percentage'] = (dist_df['Count'] / dist_df['Count'].sum()) * 100
display(dist_df)

# Xuất ra JSON nếu cần tích hợp hệ thống khác
# print(dist_df.to_json(orient='records', indent=4))

In [ ]:
import numpy as np

# 1. Tạo cột lương trung bình (nếu chưa dùng normalized_salary)
# Logic: Nếu có med_salary thì lấy, không thì lấy (min + max) / 2
def calculate_avg_salary(row):
    if not pd.isna(row['med_salary']):
        return row['med_salary']
    elif not pd.isna(row['min_salary']) and not pd.isna(row['max_salary']):
        return (row['min_salary'] + row['max_salary']) / 2
    else:
        return np.nan

df_posting['avg_salary_raw'] = df_posting.apply(calculate_avg_salary, axis=1)

# 2. Chuẩn hóa về Lương Năm (Yearly) dựa trên pay_period
# Kiểm tra các giá trị trong pay_period
print("Các loại kỳ hạn trả lương:", df_posting['pay_period'].unique())

def advanced_salary_normalization(row):
    salary = row['avg_salary_raw']
    period = str(row['pay_period']).upper()
    work_type = str(row['work_type']).upper()

    if pd.isna(salary): return np.nan

    # Xác định số giờ làm việc giả định
    hours_per_week = 40
    if 'PART' in work_type:
        hours_per_week = 20 # Giả định Part-time làm 20h

    if 'HOUR' in period:
        return salary * hours_per_week * 52
    elif 'MONTH' in period:
        return salary * 12
    elif 'WEEK' in period:
        return salary * 52
    elif 'BIWEEK' in period: # Đã xử lý text contains WEEK
        return salary * 26
    return salary

df_posting['yearly_salary'] = df_posting.apply(advanced_salary_normalization, axis=1)

# 3. Lọc dữ liệu rác và chỉ lấy USD (quan trọng vì dataset global)
df_clean = df_posting[
    (df_posting['yearly_salary'].notnull()) &
    (df_posting['currency'] == 'USD') &
    (df_posting['yearly_salary'] > 1000) # Lọc lương quá thấp (lỗi nhập liệu)
]

print(f"Số lượng bản ghi sạch sau khi xử lý: {len(df_clean)}")
print(df_clean[['title', 'formatted_experience_level', 'yearly_salary']].head())

In [ ]:
# --- TINH CHỈNH DỮ LIỆU: LOẠI BỎ NGOẠI LAI (OUTLIERS) ---
# Loại bỏ các mức lương trên 600,000 USD/năm (thường là lỗi nhập liệu hoặc outliers cực đoan)
df_clean = df_clean[df_clean['yearly_salary'] <= 600000]

print(f"Số lượng bản ghi sau khi lọc sạch outliers (>600k): {len(df_clean)}")

# Kiểm tra lại xem Max lương của Houston đã về mức hợp lý chưa
print("\nKiểm tra lại lương tại Houston, TX sau khi lọc:")
display(df_clean[df_clean['location'] == 'Houston, TX']['yearly_salary'].describe())

In [ ]:
# Kiểm tra phân bố các cột định danh quan trọng
print("Experience Level:", df_posting['formatted_experience_level'].value_counts())
print("\nWork Type:", df_posting['work_type'].value_counts())
print("\nTop 10 Job Titles:", df_posting['title'].value_counts().head(10))

In [ ]:
# Sửa lại lương cho các bản ghi BIWEEKLY (nếu có)
# Logic cũ nhân 52, logic đúng là nhân 26. Vì vậy ta chia 2.
mask_biweekly = df_clean['pay_period'].astype(str).str.contains('BIWEEKLY', case=False, na=False)
df_clean.loc[mask_biweekly, 'yearly_salary'] = df_clean.loc[mask_biweekly, 'yearly_salary'] / 2

print("Đã cập nhật lại lương cho nhóm Bi-weekly.")

In [ ]:
# --- 2. THỐNG KÊ LƯƠNG THEO KINH NGHIỆM (Dạng Bảng) ---
print("### 2. Chi tiết Lương theo Cấp bậc Kinh nghiệm ###")

# Thứ tự cấp bậc mong muốn
order_exp = ['Internship', 'Entry level', 'Associate', 'Mid-Senior level', 'Director', 'Executive']

# Groupby và tính toán các chỉ số quan trọng: Count, Mean, Median (50%), Q1 (25%), Q3 (75%)
exp_stats = df_clean.groupby('formatted_experience_level')['yearly_salary'].describe()

# Sắp xếp lại theo thứ tự logic
exp_stats = exp_stats.reindex(order_exp)

# Hiển thị bảng
display(exp_stats)

# Gợi ý: Nếu bạn muốn biết mức lương phổ biến nhất (Mode)
mode_salary = df_clean.groupby('formatted_experience_level')['yearly_salary'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
print("\nLương phổ biến nhất (Mode) theo từng cấp bậc:")
print(mode_salary)

In [ ]:
# --- 3. TOP 15 CÔNG VIỆC LƯƠNG CAO NHẤT (Dạng Bảng) ---
# (Sử dụng lại logic tính toán của bạn)
job_stats = df_clean.groupby('title').agg(
    median_salary=('yearly_salary', 'median'),
    posting_count=('title', 'count')
).reset_index()

min_postings = 10
popular_jobs = job_stats[job_stats['posting_count'] >= min_postings]
top_paying_jobs = popular_jobs.sort_values(by='median_salary', ascending=False).head(15)

print(f"### Top 15 Công việc Lương cao nhất (Min {min_postings} postings) ###")
# Reset index để hiển thị thứ hạng 1, 2, 3...
display(top_paying_jobs.reset_index(drop=True))

# Xuất dữ liệu dạng JSON để AI dễ đọc
print("\n--- JSON Output (Dành cho xử lý tự động) ---")
print(top_paying_jobs.to_json(orient='records', indent=2))

In [ ]:
# --- 4. SO SÁNH REMOTE vs ON-SITE & ĐỊA ĐIỂM (Dạng Bảng) ---
# 4.1. So sánh Remote vs Non-Remote
# Giả sử bạn có cột 'work_mode' hoặc 'remote_allowed'
# Nếu chưa có work_mode, ta tạo tạm
# Loại bỏ location chung chung
df_clean = df_clean[df_clean['location'] != 'United States']
df_clean['is_remote'] = df_clean['remote_allowed'].apply(lambda x: 'Remote' if x == 1 else 'On-site')

remote_stats = df_clean.groupby('is_remote')['yearly_salary'].agg(['count', 'mean', 'median', 'std'])
print("### So sánh Lương: Remote vs On-site ###")
display(remote_stats)

# 4.2. Top 10 Địa điểm
top_locations = df_clean['location'].value_counts().head(10).index
df_loc = df_clean[df_clean['location'].isin(top_locations)]

loc_stats = df_loc.groupby('location')['yearly_salary'].describe().sort_values(by='50%', ascending=False)
print("\n### Thống kê Lương tại Top 10 Địa điểm ###")
display(loc_stats)

In [ ]:
import re
# --- XLDL: PHÂN TÍCH DỮ LIỆU DẠNG BẢNG & JSON (THAY THẾ BIỂU ĐỒ) ---

import pandas as pd
import numpy as np

# 1. Thống kê chi tiết Lương theo Kinh nghiệm (Thay thế Boxplot)
# Chúng ta sẽ tính các chỉ số percentile (25%, 50%, 75%) tương ứng với các vạch của Boxplot
order_exp = ['Internship', 'Entry level', 'Associate', 'Mid-Senior level', 'Director', 'Executive']

# Group và tính toán
exp_analysis = df_clean.groupby('formatted_experience_level')['yearly_salary'].describe(
    percentiles=[.25, .50, .75]
).reindex(order_exp)

print("### 1. Bảng dữ liệu phân bố Lương theo Kinh nghiệm (Thay thế Boxplot) ###")
display(exp_analysis)

print("\n--- JSON Output (Experience vs Salary) ---")
# Xuất JSON để dễ dàng parse nếu cần
print(exp_analysis[['count', 'mean', '25%', '50%', '75%']].to_json(orient='index', indent=2))


# 2. Phân tích từ khóa kỹ năng (Thay thế Barplot)
target_skills = ['PYTHON', 'SQL', 'EXCEL', 'TABLEAU', 'POWER BI', 'AWS', 'JAVA', 'PROJECT MANAGEMENT', 'SPARK', 'HADOOP']

target_skills_refined = {
    'PYTHON': r'\bPYTHON\b',
    'JAVA': r'\bJAVA\b', # Không bắt Javascript
    'SQL': r'\bSQL\b',
    'EXCEL': r'\bEXCEL\b',
    'C++': r'\bC\+\+\b', # Ký tự đặc biệt
    'AWS': r'\bAWS\b|AMAZON WEB SERVICES'
}

def extract_skills_regex(text, skill_dict):
    if pd.isna(text): return []
    text_upper = text.upper()
    found = []
    for skill_name, pattern in skill_dict.items():
        if re.search(pattern, text_upper):
            found.append(skill_name)
    return found

# Áp dụng hàm
df_clean['extracted_skills'] = df_clean['description'].apply(lambda x: extract_skills_regex(x, target_skills_refined))

# Đếm tần suất (Flatten list -> Series -> Value counts)
all_skills = [skill for sublist in df_clean['extracted_skills'] for skill in sublist]
skill_counts = pd.Series(all_skills).value_counts().reset_index()
skill_counts.columns = ['Skill', 'Count']
skill_counts['Percentage'] = (skill_counts['Count'] / len(df_clean) * 100).round(2)

print("\n### 2. Bảng Tần suất Kỹ năng (Thay thế Barplot) ###")
display(skill_counts)

print("\n--- JSON Output (Top Skills) ---")
print(skill_counts.to_json(orient='records', indent=2))

In [ ]:
# --- XLDL: PHÂN TÍCH TÁC ĐỘNG CỦA KỸ NĂNG LÊN MỨC LƯƠNG ---

# Danh sách kỹ năng (giữ nguyên như trước)
target_skills = ['PYTHON', 'SQL', 'EXCEL', 'TABLEAU', 'POWER BI', 'AWS', 'JAVA', 'PROJECT MANAGEMENT']

# Tạo một list để lưu kết quả
skill_salary_stats = []

for skill in target_skills:
    # Lọc các bản ghi có chứa kỹ năng này trong cột extracted_skills
    # Lưu ý: extracted_skills là list, nên ta dùng apply để kiểm tra
    mask = df_clean['extracted_skills'].apply(lambda x: skill in x)

    if mask.sum() > 0: # Chỉ tính nếu có dữ liệu
        stats = df_clean[mask]['yearly_salary'].describe()
        skill_salary_stats.append({
            'Skill': skill,
            'Job_Count': stats['count'],
            'Median_Salary': stats['50%'], # Mức lương trung vị
            'Mean_Salary': stats['mean'],
            'Top_25%_Salary': stats['75%'] # Mức lương cao (Q3)
        })

# Chuyển thành DataFrame và sắp xếp theo Lương trung vị giảm dần
df_skill_salary = pd.DataFrame(skill_salary_stats).sort_values(by='Median_Salary', ascending=False)

print("### 3. Mức lương trung vị theo từng Kỹ năng (Xếp hạng giá trị) ###")
display(df_skill_salary)

# Xuất JSON để dễ nhìn
print("\n--- JSON Output (Skill ROI) ---")
print(df_skill_salary.to_json(orient='records', indent=2))